## Imports

In [1]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
#from tensorflow.keras.applications.resnet50 import ResNet50
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds

print("Tensorflow version " + tf.__version__)

import tensorflow as tf
from tensorflow.keras.applications import ResNet50

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam



Tensorflow version 2.15.1


## Parameters

In [2]:

IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 12
DATASET_DIR = "D:\projects\Amin_Daghig_carmodel_res50\model1_classes"

In [3]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs:", tf.config.list_physical_devices("GPU"))

TF version: 2.15.1
Built with CUDA: False
GPUs: []


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [5]:
!nvidia-smi

Tue Aug 19 11:12:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.97                 Driver Version: 580.97         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P8              4W /   35W |     552MiB /   4096MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
#loading in rgb we can grayscale and do agumentation also

(val_ds2 , train_ds2) = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

Found 3037 files belonging to 12 classes.
Using 2430 files for training.
Using 607 files for validation.


In [5]:
import numpy as np

y_train = np.concatenate([labels.numpy() for _, labels in train_ds2], axis=0)


In [6]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

In [7]:
class_weights = dict(zip(np.unique(y_train), class_weights))
print(class_weights)

{0: 1.806547619047619, 1: 0.5214776632302406, 2: 0.6485042735042735, 3: 0.9367283950617284, 4: 0.6021825396825397, 5: 1.297008547008547, 6: 1.2645833333333334, 7: 2.529166666666667, 8: 1.5328282828282829, 9: 1.631720430107527, 10: 0.6569264069264069, 11: 1.9455128205128205}


In [ ]:
'''
data_augmentation = Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1)
])

'''


data_augmentation = Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.3),
    tf.keras.layers.RandomContrast(0.3),
    tf.keras.layers.RandomBrightness(factor=0.3)
])


In [10]:
from tensorflow.keras.applications import EfficientNetB0


base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model initially
for layer in base_model.layers:
    layer.trainable = False


In [11]:
inputs = base_model.input
x = data_augmentation(inputs)               # augmentation before EfficientNet
x = base_model(x, training=False)           # pass through EfficientNet
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(64, activation="relu")(x)
outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs, outputs)

In [ ]:
'''
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
predictions = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)
'''

In [12]:
model.compile(optimizer="Adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [13]:
history = model.fit(   #no defined LR
    train_ds2,
    validation_data=val_ds2,
    epochs=10,
    verbose=1,
    class_weight=class_weights
)

Epoch 1/10


19/19 [==============================] - 58s 3s/step - loss: 2.5083 - accuracy: 0.0988 - val_loss: 2.3596 - val_accuracy: 0.1959
Epoch 2/10
19/19 [==============================] - 53s 3s/step - loss: 2.2503 - accuracy: 0.2372 - val_loss: 2.2595 - val_accuracy: 0.2609
Epoch 3/10
19/19 [==============================] - 52s 3s/step - loss: 2.0150 - accuracy: 0.3213 - val_loss: 2.1332 - val_accuracy: 0.2868
Epoch 4/10
19/19 [==============================] - 54s 3s/step - loss: 1.8661 - accuracy: 0.3674 - val_loss: 1.9779 - val_accuracy: 0.3630
Epoch 5/10
19/19 [==============================] - 54s 3s/step - loss: 1.6910 - accuracy: 0.4514 - val_loss: 1.9088 - val_accuracy: 0.3922
Epoch 6/10
19/19 [==============================] - 54s 3s/step - loss: 1.5770 - accuracy: 0.5124 - val_loss: 1.7937 - val_accuracy: 0.4535
Epoch 7/10
19/19 [==============================] - 50s 3s/step - loss: 1.4934 - accuracy: 0.5124 - val_loss: 1.6920 - val_accuracy: 0.4720
Epoch 8/10
19/19 [

In [14]:
#second stage trasining
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(optimizer="Adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [16]:
history_finetune = model.fit(
    train_ds2,
    validation_data=val_ds2,
    epochs=12,
    verbose=1
)

Epoch 1/12
19/19 [==============================] - 56s 3s/step - loss: 1.0740 - accuracy: 0.6458 - val_loss: 1.3572 - val_accuracy: 0.5815
Epoch 2/12
19/19 [==============================] - 57s 3s/step - loss: 0.7575 - accuracy: 0.7644 - val_loss: 1.0537 - val_accuracy: 0.6708
Epoch 3/12
19/19 [==============================] - 57s 3s/step - loss: 0.6721 - accuracy: 0.7792 - val_loss: 0.9566 - val_accuracy: 0.7111
Epoch 4/12
19/19 [==============================] - 57s 3s/step - loss: 0.4964 - accuracy: 0.8501 - val_loss: 0.9307 - val_accuracy: 0.7366
Epoch 5/12
19/19 [==============================] - 57s 3s/step - loss: 0.4553 - accuracy: 0.8567 - val_loss: 0.8810 - val_accuracy: 0.7403
Epoch 6/12
19/19 [==============================] - 58s 3s/step - loss: 0.3496 - accuracy: 0.8781 - val_loss: 0.8578 - val_accuracy: 0.7560
Epoch 7/12
19/19 [==============================] - 58s 3s/step - loss: 0.3002 - accuracy: 0.9012 - val_loss: 0.9032 - val_accuracy: 0.7523
Epoch 8/12
19/19 [==

In [17]:
model.save("efficientNetB0_13class_v7.h5")

c:\anaconda\envs\yolov7-gpu-env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
